In [ ]:
from torch import nn
import os
import torch

In [ ]:
cfg = [32, 'M', 64, 'M', 128, 64, 128, 'M', 256, 128, 256, 'M',
       512, 256, 512, 256, 512, 'M', 1024, 512, 1024, 512, 1024]

In [ ]:
def make_layers(cfg, in_channels=3, batch_norm=True):
    """
    从配置参数中构建网络
    :param cfg:  参数配置
    :param in_channels: 输入通道数,RGB彩图为3, 灰度图为1
    :param batch_norm:  是否使用批正则化
    :return:
    """
    layers = []
    flag = True             # 用于变换卷积核大小,(True选后面的,False选前面的)
    in_channels= in_channels
    for v in cfg:
        if v == 'M':
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else:
            layers.append(nn.Conv2d(in_channels = in_channels,
                                   out_channels= v,
                                   kernel_size=(1, 3)[flag],
                                   stride=1,
                                   padding=(0,1)[flag],
                                   bias=False))
            if batch_norm:
                layers.append(nn.BatchNorm2d(v))
            in_channels = v

            layers.append(nn.LeakyReLU(negative_slope=0.1, inplace=True))

        flag = not flag

    return nn.Sequential(*layers)

In [ ]:
class Darknet19(nn.Module):
    """
    Darknet19 模型
    """
    def __init__(self, num_classes=1000, in_channels=3, batch_norm=True, pretrained=False):
        """
        模型结构初始化
        :param num_classes: 最终分类数       (nums of classification.)
        :param in_channels: 输入数据的通道数  (input pic`s channel.)
        :param batch_norm:  是否使用正则化    (use batch_norm, True or False;True by default.)
        :param pretrained:  是否导入预训练参数 (use the pretrained weight)
        """
        super(Darknet19, self).__init__()
        # 调用nake_layers 方法搭建网络
        # (build the network)
        self.features = make_layers(cfg, in_channels=in_channels, batch_norm=batch_norm)
        # 网络最后的分类层,使用 [1x1卷积和全局平均池化] 代替全连接层.
        # (use 1x1 Conv and averagepool replace the full connection layer.)
        self.classifier = nn.Sequential(
            nn.Conv2d(1024, num_classes, kernel_size=1, stride=1),
            nn.AdaptiveAvgPool2d(output_size=(1)),
            nn.Softmax(dim=0)
        )
        # 导入预训练模型或初始化
        if pretrained:
            self.load_weight()
        else:
            self._initialize_weights()

    def forward(self, x):
        # 前向传播
        x = self.features(x)
        x = self.classifier(x)
        # 为了使输出与使用全连接层格式相同,这里对输出进行了resize
        # resize [B, num_classes, 1, 1] to [B, num_classes]
        x = x.view(x.size(0),-1)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def load_weight(self):
        weight_file = 'weights/darknet19-deepBakSu-e1b3ec1e.pth'
        if not os.path.exists(weight_file):
            import wget

            url = 'https://s3.ap-northeast-2.amazonaws.com/deepbaksuvision/darknet19-deepBakSu-e1b3ec1e.pth'
            print('将下载权重文件,如果太慢,你可以自己下载然后放到weights文件夹下'.format(url))
            print('Will download weight file from {}'.format(url))
            wget.download(url=url, out='weights/darknet19-deepBakSu-e1b3ec1e.pth')
        # 转换权重文件中的keys.(change the weights dict `keys)
        assert len(torch.load(weight_file).keys()) == len(self.state_dict().keys())
        dic = {}
        for now_keys, values in zip(self.state_dict().keys(), torch.load(weight_file).values()):
            dic[now_keys]=values
        self.load_state_dict(dic)

In [ ]:
net = Darknet19(num_classes=1000, pretrained=True)

In [ ]:
print(net)

In [ ]:
net.features[0].weight

In [18]:
from collections import OrderedDict
from torch import nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from base import BaseModel

model_paths = {
    'darknet19': 'https://s3.ap-northeast-2.amazonaws.com/deepbaksuvision/darknet19-deepBakSu-e1b3ec1e.pth'
}

class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        N = x.data.size(0)
        C = x.data.size(1)
        H = x.data.size(2)
        W = x.data.size(3)
        x = F.avg_pool2d(x, (H, W))
        x = x.view(N, C)
        return x


class Darknet19(BaseModel):
    def __init__(self, pretrained=True):
        super(Darknet19, self).__init__()

        self.features = nn.Sequential(OrderedDict([
        ('layer1', nn.Sequential(OrderedDict([
            ('conv1_1', nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn1_1', nn.BatchNorm2d(32)),
            ('leaky1_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer2', nn.Sequential(OrderedDict([
            ('conv2_1', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn2_1', nn.BatchNorm2d(64)),
            ('leaky2_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer3', nn.Sequential(OrderedDict([
            ('conv3_1', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_1', nn.BatchNorm2d(128)),
            ('leaky3_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_2', nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn3_2', nn.BatchNorm2d(64)),
            ('leaky3_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_3', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_3', nn.BatchNorm2d(128)),
            ('leaky3_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool3', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer4', nn.Sequential(OrderedDict([
            ('conv4_1', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_1', nn.BatchNorm2d(256)),
            ('leaky4_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_2', nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn4_2', nn.BatchNorm2d(128)),
            ('leaky4_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_3', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_3', nn.BatchNorm2d(256)),
            ('leaky4_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool4', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer5', nn.Sequential(OrderedDict([
            ('conv5_1', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_1', nn.BatchNorm2d(512)),
            ('leaky5_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_2', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn5_2', nn.BatchNorm2d(256)),
            ('leaky5_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_3', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_3', nn.BatchNorm2d(512)),
            ('leaky5_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_4', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn5_4', nn.BatchNorm2d(256)),
            ('leaky5_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_5', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_5', nn.BatchNorm2d(512)),
            ('leaky5_5', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool5', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer6', nn.Sequential(OrderedDict([
            ('conv6_1', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_1', nn.BatchNorm2d(1024)),
            ('leaky6_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_2', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn6_2', nn.BatchNorm2d(512)),
            ('leaky6_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_3', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_3', nn.BatchNorm2d(1024)),
            ('leaky6_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_4', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn6_4', nn.BatchNorm2d(512)),
            ('leaky6_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_5', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_5', nn.BatchNorm2d(1024)),
            ('leaky6_5', nn.LeakyReLU(0.1, inplace=True))
        ])))
        ]))

        self.classifier = nn.Sequential(OrderedDict([
        ('conv7_1', nn.Conv2d(1024, 1000, kernel_size=(1, 1), stride=(1, 1))),
        ('globalavgpool', GlobalAvgPool2d()),
        ('softmax', nn.Softmax(dim=1))
        ]))

        if pretrained:
            # self.load_state_dict(model_zoo.load_url(model_paths['darknet19'],  progress=True))
            self.load_weight()
            print('Model is loaded')

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        return out
    def load_weight(self):
        weight_file = 'weights/darknet19-deepBakSu-e1b3ec1e.pth'
        if not os.path.exists(weight_file):
            import wget

            url = 'https://s3.ap-northeast-2.amazonaws.com/deepbaksuvision/darknet19-deepBakSu-e1b3ec1e.pth'
            print('将下载权重文件,如果太慢,你可以自己下载然后放到weights文件夹下'.format(url))
            print('Will download weight file from {}'.format(url))
            wget.download(url=url, out='weights/darknet19-deepBakSu-e1b3ec1e.pth')
        # 转换权重文件中的keys.(change the weights dict `keys)
        assert len(torch.load(weight_file).keys()) == len(self.state_dict().keys())
        dic = {}
        for now_keys, values in zip(self.state_dict().keys(), torch.load(weight_file).values()):
            dic[now_keys]=values
        self.load_state_dict(dic)
        print('Weights are loaded!')

In [19]:
model = Darknet19(True)

Weights are loaded!
Model is loaded


In [26]:
model.features[0][0].weight

Parameter containing:
tensor([[[[-5.1238e-02,  1.1857e-02,  1.6812e-02],
          [-1.3126e-01,  2.4813e-02,  8.2779e-02],
          [-9.3807e-03,  3.7781e-02,  1.0434e-02]],

         [[-4.9496e-02,  4.2385e-02,  5.7809e-02],
          [-1.4099e-01,  3.2534e-02,  9.5713e-02],
          [-4.5277e-02,  2.1831e-02,  1.0215e-02]],

         [[-1.8086e-02,  1.5441e-02,  1.7712e-02],
          [-9.7840e-02,  3.9564e-03,  5.2311e-02],
          [ 7.1512e-03,  5.0424e-03, -1.1215e-02]]],


        [[[ 1.7984e-02,  4.6606e-02,  5.5725e-02],
          [ 3.1353e-02, -3.4171e-02, -3.0916e-02],
          [ 3.6990e-02, -6.7037e-02, -2.4502e-02]],

         [[ 2.1863e-02,  3.1940e-02,  4.4470e-02],
          [ 1.7214e-02, -9.2305e-02, -9.1293e-02],
          [ 2.4250e-02, -1.1196e-01, -6.8886e-02]],

         [[-4.9606e-02, -5.6575e-02, -2.5855e-02],
          [-5.6195e-02, -8.9756e-02, -6.1751e-02],
          [-9.2584e-03, -6.1846e-02, -1.5014e-02]]],


        [[[ 1.2972e-01, -1.4389e-01, -7.3945